In [2]:
import requests
import os
import json
import time
import pyodbc 


bearer_token = "AAAAAAAAAAAAAAAAAAAAAI3lZwEAAAAAeqEmDP8s%2FEHuzbpIwc2JxWtN%2Brg%3Dca66hTTDI7Gqtsj0MrtQ8Ap58pCBLjgER6aKeDNWZt6cIRW6oB"


def create_url(user_id, next_token=''):
    if next_token == '':
        return "https://api.twitter.com/2/users/{}/followers".format(user_id)
    else:
        return "https://api.twitter.com/2/users/{}/followers?pagination_token={}".format(user_id, next_token)


def get_params():
    return {"user.fields": "created_at"}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def connect_to_endpoint_id(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def get_id(user):
    
    if user[-1] == '\n':
        user = user[:-1]
    
    usernames = "usernames=" + str(user)
    user_fields = "user.fields=description,created_at"

    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    # print(connect_to_endpoint_id(url))
    json_response = connect_to_endpoint_id(url)['data'][0]
    return int(json_response['id'])


def save_in_db(user, data):
    print("save in db")
    cnxn = pyodbc.connect(driver='{SQL Server}', server='MERSAD-PC1', database='Twitter-ph2', trusted_connection='yes')
    cursor = cnxn.cursor()
    
    for i in data:
        uname = i['username']
        query_str = f'INSERT INTO [dbo].[TwitterAPIFollowers] VALUES {uname, user};'
        cursor.execute(query_str)
    
    cnxn.commit()
    cnxn.close()
    
    

def read_token_from_file(file_name):
    with open(file_name, "r") as f:
        line = f.readline()
        if line != "":
            print("Token Found: " + line)
            return True, line
        else:
            print("No PreToken")
            return True, ""

        
def write_token_in_file(file_name, token):
    with open(file_name, "w") as f:
        f.write(token)


def read_users_from_file(file_name):
    result = []
    with open(file_name, "r") as f:
        lines = f.readlines()
        
        for line in lines:
            if line[-1] != "*":
                result.append(line)
        
    return result

def remove_finished_user(file_name, username):
    lines = []
    with open(file_name, "r") as f:
        lines = f.readlines()
        
    for index, line in enumerate(lines):
        if line == username:
            lines[index] = lines[index] + " *"
            
    with open(file_name, "w") as f:
        f.writelines(lines)

def main():
    users = read_users_from_file("follower_users.txt")
    
    for user in users:
        print("user:", user)
        user_id = get_id(user)
        
        json_response = []
        new_token, next_token = read_token_from_file("follower_token.txt")
        count = 0
        while new_token == True:
            count += 1
            url = create_url(user_id, next_token)
            params = get_params()
            # json_response = connect_to_endpoint(url, params)
            # print(json_response)

            json_response.append(connect_to_endpoint(url, params))

            try:
                if json_response[-1]['meta']['next_token']:
                    next_token = json_response[-1]['meta']['next_token']
                else:
                    new_token = False
                    next_token = ""
                    write_token_in_file("follower_token.txt", next_token)
                    remove_finished_user("follower_users.txt", user)
            except:
                print("Error")
                new_token = False
                next_token = ""
                write_token_in_file("follower_token.txt", next_token)
                remove_finished_user("follower_users.txt", user)

                
            if count % 15 == 0:
                for i in json_response:
                    save_in_db(user, i['data'])
            
                json_response = []
                
                write_token_in_file("follower_token.txt", next_token)
                
                time.sleep(15*60)
                
        
        time.sleep(15*60)




if __name__ == "__main__":
    main()

user: al_honini

200
Token Found: RE1KD7A8ARJ16ZZZ
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
2

save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
save in db
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
save in db
save in